In [ ]:
import cv2
import time
import PIL.Image
from io import BytesIO
import IPython.display
import ipywidgets as widgets
import threading

In [ ]:
def hsv_select_live(limit = 10):
    # Mask and display video
    display = IPython.display.display('', display_id=1)
    
    # Create sliders
    h = widgets.IntRangeSlider(value=[0, 179], min=0, max=179, description='Hue:', continuous_update=True, layout=widgets.Layout(width='100%'))
    s = widgets.IntRangeSlider(value=[0, 255], min=0, max=255, description='Saturation:', continuous_update=True, layout=widgets.Layout(width='100%'))
    v = widgets.IntRangeSlider(value=[0, 255], min=0, max=255, description='Value:', continuous_update=True, layout=widgets.Layout(width='100%'))
    display.update(h)
    display.update(s)
    display.update(v)
    
    # Live masked video for the thread
    def show_masked_video():
        video = cv2.VideoCapture(0)
        start = time.time()
        while time.time() - start < limit:
            frame = video.read()[1]
            if frame is not None:
                hsv_min = (h.value[0], s.value[0], v.value[0])
                hsv_max = (h.value[1], s.value[1], v.value[1])
                frame = cv2.resize(frame, (320, 240))
                frame = cv2.flip(frame, 1)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img_hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
                mask = cv2.inRange(img_hsv, hsv_min, hsv_max)
                img_masked = cv2.bitwise_and(frame, frame, mask = mask)
                f = BytesIO()
                PIL.Image.fromarray(img_masked).save(f, 'jpeg')
                img_jpeg = IPython.display.Image(data=f.getvalue())
                display.update(img_jpeg)
                time.sleep(0.25)
        video.release()
    hsv_thread = threading.Thread(target=show_masked_video)
    hsv_thread.start()